# 条件随机场

条件随机场（Conditional Random Fields，CRF）和隐马尔可夫模型很像，后者的基础是状态矩阵，转移矩阵和观测矩阵，前者是观测序列和标注序列——对应于状态概率和“观测”概率。整体上这是一系列模型的演化

![](https://raw.githubusercontent.com/LibertyDream/diy_img_host/master/img/2019-12-05_bayes_logistic_HMM_CRF.png)

条件随机场是判别模型，最大特点是可以表示成概率无向图。无向图上结点间的连接关系表示了随机变量集合间的条件独立性，即马尔可夫性。比如，下图中给定随机变量组 $Y_C$ 条件下随机变量组 $Y_A$, $Y_B$ 是条件独立的，这个图就是具有马尔可夫性的。
$$
P\left(Y_{A}, Y_{B} | Y_{C}\right)=P\left(Y_{A} | Y_{C}\right) P\left(Y_{B} | Y_{C}\right)
$$


![](https://raw.githubusercontent.com/LibertyDream/diy_img_host/master/img/2019-12-05_markov_property.png)

条件随机场中“随机场”就是指能构成这种图，而“条件”是说在给定输入 $X$ 时，输出 $Y$ 可以形成随机场

这里只说线性链条件随机场，即输入输出都是“链子”形式的序列（观测序列和标记序列），两条“链子”能构成条件随机场

![](https://raw.githubusercontent.com/LibertyDream/diy_img_host/master/img/2019-12-05_Linear-Chain_Conditional_Random_Fields.png)

模型的参数形式为
$$
P(y | x)=\frac{1}{Z(x)} \exp \left(\sum_{i, k} \lambda_{k} t_{k}\left(y_{i-1}, y_{i}, x, i\right)+\sum_{i, l} \mu_{l} s_{l}\left(y_{i}, x, i\right)\right)\\
Z(x)=\sum_{y} \exp \left(\sum_{i, k} \lambda_{k} t_{k}\left(y_{i-1}, y_{i}, x, i\right)+\sum_{i, l} \mu_{l} s_{l}\left(y_{i}, x, i\right)\right)
$$
简化为
$$
\begin{aligned} P(y | x) &=\frac{1}{Z(x)} \exp \sum_{k=1}^{K} w_{k} f_{k}(y, x) \\ Z(x) &=\sum_{y} \exp \sum_{k=1}^{K} w_{k} f_{k}(y, x) \\
f_{k}(y, x)&=\sum_{i=1}^{n} f_{k}\left(y_{i-1}, y_{i}, x, i\right), \quad k=1,2, \cdots, K\\
f_{k}\left(y_{i-1}, y_{i}, x, i\right)&=\left\{\begin{array}{ll}{t_{k}\left(y_{i-1}, y_{i}, x, i\right),} & {k=1,2, \cdots, K_{1}} \\ {s_{l}\left(y_{i}, x, i\right),} & {k=K_{1}+l ; l=1,2, \cdots, K_{2}}\end{array}\right.\\
w_{k}&=\left\{\begin{array}{ll}{\lambda_{k},} & {k=1,2, \cdots, K_{1}} \\ {\mu_{l},} & {k=K_{1}+l ; l=1,2, \cdots, K_{2}}\end{array}\right.
\end{aligned}
$$

- 概率计算问题

给定条件随机场 $P(Y|X)$，获得输入序列 $x$ 和输出序列 $y$，求其中的条件概率和期望。解法是前向-后向算法

- 条件随机场的学习问题

给定训练数据集，估计模型参数。条件随机场模型实际上是定义在时序数据上的对数线性模型，通过极大似然估计求解，具体实现可以是拟牛顿法或梯度下降
$$
P_{w}(y | x)=\frac{\exp \left(\sum_{i=1}^{n} w_{i} f_{i}(x, y)\right)}{\sum_{y} \exp \left(\sum_{i=1}^{n} w_{i} f_{i}(x, y)\right)}\\
L(w)=L_{\tilde{P}}\left(P_{w}\right)=\log \prod_{x, y} P_{w}(y | x)^{\tilde{P}(x, y)}=\sum_{x, y} \tilde{P}(x, y) \log P_{w}(y | x)\\
\begin{aligned} L(w) &=\sum_{x, y} \tilde{P}(x, y) \log P_{w}(y | x) \\ &=\sum_{x, y}\left[\tilde{P}(x, y) \sum_{k=1}^{K} w_{k} f_{k}(y, x)-\tilde{P}(x, y) \log Z_{w}(x)\right] \\ &=\sum_{j=1}^{N} \sum_{k=1}^{K} w_{k} f_{k}\left(y_{j}, x_{j}\right)-\sum_{j=1}^{N} \log Z_{w}\left(x_{j}\right) \end{aligned}
$$
目标函数
$$
\min _{w \in \mathbb{R}^{n}} f(w)=\sum_{x} \tilde{P}(x) \log \sum_{y} \exp \left(\sum_{i=1}^{n} w_{i} f_{i}(x, y)\right)-\sum_{x, y} \tilde{P}(x, y) \sum_{i=1}^{n} w_{i} f_{i}(x, y)
$$
梯度
$$
g(w)=\sum_{x, y} \tilde{P}(x) P_{w}(y | x) f(x, y)-E_{\tilde{P}}(f)
$$

- 预测问题

给定条件随机场和输入序列，求条件概率最大的输出序列，即**对观测序列进行标注**，这是该模型的一个重要应用。
$$
\begin{aligned} y^{*} &=\arg \max _{y} P_{w}(y | x) \\ &=\arg \max _{y} \frac{\exp (w \cdot F(y, x))}{Z_{w}(x)} \\ &=\arg \max _{y} \exp (w \cdot F(y, x)) \\ &=\arg \max _{y}(w \cdot F(y, x)) \end{aligned}
$$
可见就是求 $\max _{y}(w \cdot F(y, x)) $，写成路径形式
$$
\begin{array}{c}{w=\left(w_{1}, w_{2}, \cdots, w_{K}\right)^{\mathrm{T}}} \\ {F(y, x)=\left(f_{1}(y, x), f_{2}(y, x), \cdots, f_{K}(y, x)\right)^{\mathrm{T}}} \\ {f_{k}(y, x)=\sum_{i=1}^{n} f_{k}\left(y_{i-1}, y_{i}, x, i\right), \quad k=1,2, \cdots, K}\\
\max _{y} \sum_{i=1}^{n} w \cdot F_{i}\left(y_{i-1}, y_{i}, x\right)
\end{array}
$$
具体方法是维特比算法，贪心策略，从起点开始每次前往概率最大的结点，同时记录路径。
$$
\begin{aligned} \delta_{i}(l)=& \max _{1 \leqslant j \leqslant m}\left\{\delta_{i-1}(j)+w \cdot F_{i}\left(y_{i-1}=j, y_{i}=l, x\right)\right\}, \quad l=1,2, \cdots, m \\ \Psi_{i}(l)=& \arg \max _{1 \leqslant j \leqslant m}\left\{\delta_{i-1}(j)+w \cdot F_{i}\left(y_{i-1}=j, y_{i}=l, x\right)\right\}, \quad l=1,2, \cdots, m \end{aligned}
$$


# 算法实现

参见[线性链条件随机场](https://github.com/lancifollia/crf)

---

**作者：** Daniel Meng

**GitHub：**[LibertyDream](https://github.com/LibertyDream)

**博客：**[明月轩](https://LIbertydream.github.io)